In [1]:
import pickle
import os
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
model_path = r'/content/drive/MyDrive/Github/SeoupSeoul Bike Sharing Demand Prediction /Model Assets/xgboost_regressor_m2_0.923.pkl'
model = pickle.load(open(model_path, "rb"))

In [3]:
model.predict([[1.08347268,  0.2191097 ,  0.77469426,  1.80951305, -1.25245915,
       -0.25222928, -0.13179988, -0.17189109, -0.22775689, -5.35676351,
       -1.21876386,  1.0075761 ,  0.30465451,  1.72261177, -0.58051386,
       -0.57207755, -0.39840954, -0.40759561, -0.41669346, -0.40759561,
       -0.41215506, -0.41215506]])

array([142.75053], dtype=float32)

# Load SC

In [4]:
scaler_dumb_path = "/content/drive/MyDrive/Github/SeoupSeoul Bike Sharing Demand Prediction /Model Assets/scaler.pkl"
scaler = pickle.load(open(scaler_dumb_path, "rb"))

# User Input

In [5]:
code = '''

from datetime import datetime

# Define function to validate date format
def validate_date(date_string):
    try:
        datetime.strptime(date_string, '%d/%m/%Y')
        return True
    except ValueError:
        return False

# Function to validate float input
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Function to validate integer input
def is_integer(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

# Get input values from user
while True:
    date = input("Date (format: DD/MM/YYYY): ").strip()  # Remove leading/trailing spaces
    if validate_date(date):
        break
    else:
        print("Please enter a valid date in the format: DD/MM/YYYY")

while True:
    hour = input("Hour (0 to 23): ").strip()
    if is_integer(hour) and 0 <= int(hour) <= 23:
        hour = int(hour)
        break
    else:
        print("Please enter a valid hour between 0 and 23")

while True:
    temperature = input("Temperature(°C) {-15 to 39}: ").strip()
    if is_float(temperature) and -15 <= float(temperature) <= 39:
        temperature = float(temperature)
        break
    else:
        print("Please enter a valid temperature between -15 and 39")

while True:
    humidity = input("Humidity(%) {0 to 98}: ").strip()
    if is_float(humidity) and 0 <= float(humidity) <= 98:
        humidity = float(humidity)
        break
    else:
        print("Please enter a valid humidity percentage between 0 and 98")

while True:
    wind_speed = input("Wind speed(m/s) {0 to 7}: ").strip()
    if is_float(wind_speed) and 0 <= float(wind_speed) <= 7:
        wind_speed = float(wind_speed)
        break
    else:
        print("Please enter a valid wind speed between 0 and 7")

while True:
    visibility = input("Visibility(m) {27 to 2000}: ").strip()
    if is_integer(visibility) and 27 <= int(visibility) <= 2000:
        visibility = int(visibility)
        break
    else:
        print("Please enter a valid visibility between 27 and 2000")

while True:
    solar_radiation = input("Solar radiation {0 to 3}: ").strip()
    if is_float(solar_radiation) and 0 <= float(solar_radiation) <= 3:
        solar_radiation = float(solar_radiation)
        break
    else:
        print("Please enter a valid solar radiation between 0 and 3")

while True:
    rainfall = input("Rainfall(mm) (0 to 35): ").strip()
    if is_float(rainfall) and 0 <= float(rainfall) <= 35:
        rainfall = float(rainfall)
        break
    else:
        print("Please enter a valid rainfall between 0 and 35")

while True:
    snowfall = input("Snowfall(cm) (0 to 8): ").strip()
    if is_float(snowfall) and 0 <= float(snowfall) <= 8:
        snowfall = float(snowfall)
        break
    else:
        print("Please enter a valid snowfall between 0 and 8")

while True:
    seasons = input("Type Season Name (Spring, Summer, Autumn, Winter: ").strip().title()
    if seasons in ["Spring", "Summer", "Autumn", "Winter"]:
        break
    else:
        print("Please enter a valid season: Spring, Summer, Autumn, Winter")

while True:
    holiday = input("Type ('Holiday' or 'No Holiday'): ").strip().title()
    if holiday in ["Holiday", "No Holiday"]:
        break
    else:
        print("Please enter a valid option: Holiday or No Holiday")

while True:
    functioning_day = input("Type Is it a functioning day (Yes or No): ").strip().title()
    if functioning_day in ["Yes", "No"]:
        break
    else:
        print("Please enter a valid option: Yes or No")

# Use the input variables in further processing or analysis...

# ------------->>>


# Converting User Data into Model Consumable Format
from datetime import datetime
holiday_dic = {"No Holiday":0, "Holiday":1}
functioning_dic = {"No":0, "Yes":1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date, "%d/%m/%Y")
    return{"day": dt.day, "month": dt.month, "year":dt.year, "week_day" : dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)


# -----
u_input_list = [hour, temperature, humidity, wind_speed, visibility, solar_radiation, rainfall, snowfall,
                holiday_dic[holiday], functioning_dic[functioning_day],
                str_to_date["day"], str_to_date["month"], str_to_date["year"]]

features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year',]
df_u_input = pd.DataFrame([u_input_list], columns = features_name)


# -----

def seasons_to_df(seasons):
    seasons_cols = ['Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype = "int")

    df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)
    if seasons in seasons_cols:
      df_seasons[seasons] = 1
    return df_seasons
df_seasons = seasons_to_df(seasons)


# ----
def days_df(week_day) :
  days_names = ['Friday','Monday', 'Saturday', 'Thursday', 'Tuesday', 'Wednesday']
  days_name_data = np.zeros((1, len(days_names)),dtype="int")

  df_days = pd.DataFrame(days_name_data, columns = days_names)

  if week_day in days_names:
    df_days[week_day] = 1
  return df_days

df_days = days_df(str_to_date["week_day"])

# ----
df_for_pred = pd.concat([df_u_input, df_seasons, df_days], axis=1)


# ----
scaler_data_for_pred = scaler.transform(df_for_pred)


# Prediction in Deployment
print()
print(f"The estimated bike demand for the date {date} and time {hour} is approximately {round(model.predict(scaler_data_for_pred).tolist()[0])}.")
'''

In [6]:
# exec(code)

In [7]:
import joblib
form_path = r'/content/drive/MyDrive/Github/SeoupSeoul Bike Sharing Demand Prediction /Model Assets/form.pkl'
joblib.dump(code,form_path)

['/content/drive/MyDrive/Github/SeoupSeoul Bike Sharing Demand Prediction /Model Assets/form.pkl']

In [8]:
# load the sclaer from pwd
Form = joblib.load(form_path)

In [9]:
# exec(Form)